# Declarative DSL Style

LabOne Q's domain-specific language (DSL) is the immediate user interface to define experiments, calibrations and everything else LabOne Q specific.

Particularly when you define experiments, you can choose between the imperative and the declarative DSL style. Each one might have some distinct advantages useful for specific situations.

In this notebook, you will learn how to:
- Use the declarative DSL style to build a Ramsey experiment
- How to manipulate sections to change the experiment's behavior without having to redefine the entire experiment.

In the following, it might help you to think of your experiment as a tree: Often, its trunk is a real-time acquisition loop, which governs the averaging. It's branches are sections, which contain either more sections or pulses.

## The Imperative Style

Most of our example notebooks use the imperative DSL style to define experiments. The imperative style makes extensive use of Python's `with` statements, and the structure of the code represents the structure of the experiment.

## The Declarative Style

In the declarative DSL style, you define the constituent elements (sweeps, sections) of your experiments, and then "declare" how they interact with each other.

## Imports

In [ ]:
# Import required packages
import numpy as np

from laboneq.contrib.example_helpers.generate_device_setup import (
    generate_device_setup_qubits,
)
from laboneq.simple import *

## Device Setup and Calibration

We'll generate a device setup and logical signal lines.

In contrast to actual experiments, we do not apply a setup calibration to keep the notebook brief. To get familiar with using `Calibration`, please refer to our tutorial [here](https://docs.zhinst.com/labone_q_user_manual/core/functionality_and_concepts/02_logical_signals/tutorials/02_calibration.html).

In [ ]:
# specify the number of qubits you want to use
number_of_qubits = 2

# generate the device setup and the qubit objects using a helper function
device_setup, qubits = generate_device_setup_qubits(
    number_qubits=number_of_qubits,
    pqsc=[{"serial": "DEV10001"}],
    hdawg=[{"serial": "DEV8001", "zsync": 0, "number_of_channels": 8, "options": None}],
    shfqc=[
        {
            "serial": "DEV12001",
            "zsync": 1,
            "number_of_channels": 6,
            "readout_multiplex": 6,
            "options": None,
        }
    ],
    include_flux_lines=True,
    server_host="localhost",
    setup_name=f"my_{number_of_qubits}_fixed_qubit_setup",
)

q0, q1 = qubits[:2]

## Ramsey Experiment

We start by defining the pulses and the parameter sweep

In [ ]:
# pulse definitions
drive_pulse = pulse_library.gaussian(uid="gaussian_drive", length=700e-9, amplitude=1)
readout_pulse = pulse_library.const(uid="Readout", length=300e-9, amplitude=0.8)

# averages
n_average = 2

# sweep parameters
n_steps = 7
start_delay = 0
stop_delay = 13e-6

time_sweep = LinearSweepParameter(
    uid="time_sweep_param", start=start_delay, stop=stop_delay, count=n_steps
)

Then, we start to define the experiment.

The experiment comprises a real-time acquisition loop, which governs the averaging; and a parameter sweep, which governs the delay between the pulses.

We'll use three sections to define our experimental sequence: In the first section, we will excite the qubit, in the second section, we'll measure it, and the third section introduces some delay for the qubit thermalization.

In [ ]:
exp_ramsey = Experiment(uid="Ramsey_Experiment")
exp_ramsey.add_signal(
    "drive",
    connect_to=device_setup.logical_signal_groups["q0"].logical_signals["drive_line"],
)
exp_ramsey.add_signal(
    "measure",
    connect_to=device_setup.logical_signal_groups["q0"].logical_signals["measure_line"],
)
exp_ramsey.add_signal(
    "acquire",
    connect_to=device_setup.logical_signal_groups["q0"].logical_signals["acquire_line"],
)

acquire_loop = AcquireLoopRt(
    uid="RT_Shots",
    count=n_average,
    averaging_mode=AveragingMode.CYCLIC,
    repetition_mode=RepetitionMode.AUTO,
)
sweep = Sweep(
    uid="Ramsey_Sweep", parameters=[time_sweep], alignment=SectionAlignment.RIGHT
)

# Qubit Excitation
excitation_section = Section(uid="qubit_excitation")
excitation_section.play(signal="drive", pulse=drive_pulse)
excitation_section.delay(signal="drive", time=time_sweep)
excitation_section.play(signal="drive", pulse=drive_pulse)

# Qubit Readout
readout_section = Section(uid="readout")
readout_section.play_after = excitation_section
readout_section.play(signal="measure", pulse=readout_pulse)
readout_section.acquire(
    signal="acquire",
    handle="ramsey",
    kernel=readout_pulse,
)

# Qubit Thermalization
delay_section = Section(uid="delay", length=2e-6)
delay_section.play_after = readout_section

After having defined the constituent elements, we can put them together:

We add the acquisition loop to the experiment, the sweep to the acquisition loop, and the sections to the sweep.

In [ ]:
exp_ramsey.add(acquire_loop)

acquire_loop.add(sweep)

sweep.add(excitation_section)
sweep.add(readout_section)
sweep.add(delay_section)

### Inspect Experiment Tree

You can use the `print` command to print the experiment tree as you have just defined it:

In [ ]:
print(exp_ramsey)

### Experiment Compilation

Before you can compile (or execute) the experiment, you need to open a session that connects to the instruments (or, here, emulates that connection).

In [ ]:
session = Session(device_setup=device_setup)
session.connect(do_emulation=True)

In [ ]:
compiled_exp = session.compile(exp_ramsey)

### Inspect with Pulse Sheet Viewer

Once the experiment is compiled, we can view the pulses in a Pulse Sheet Viewer, a HTML file which shows the sections, pulses, and their relative timings.

In [ ]:
show_pulse_sheet("Ramsey_Declarative", compiled_exp)

### Changing Elements After the Experiment Definition

When you use the declarative style of the DSL, the elements of your experiment can be changed individually. Assume that you require a longer thermalization time:

In [ ]:
delay_section.length = 10e-6

In [ ]:
compiled_exp = session.compile(exp_ramsey)

show_pulse_sheet("Slower_Ramsey_Declarative", compiled_exp)

## Reusing Sections

The declarative DSL style is particularly handy when you want to re-use sections. You could, for example, define a personal gate and apply it twice.

We create an experiment from scratch, define a gate section, and use it several times across the experiment. 

In [ ]:
gate_section = Section()
gate_section.play(signal="drive", pulse=drive_pulse, amplitude=0.5)
gate_section.play(signal="drive", pulse=drive_pulse, amplitude=0.5, phase=np.pi / 2)

readout_section = Section(uid="readout")
readout_section.play(signal="measure", pulse=readout_pulse)
readout_section.acquire(
    signal="acquire",
    handle="ramsey",
    kernel=readout_pulse,
)
readout_section.reserve(signal="drive")

exp_gate = Experiment(uid="personal_experiment")
exp_gate.add_signal(
    "drive",
    connect_to=device_setup.logical_signal_groups["q0"].logical_signals["drive_line"],
)
exp_gate.add_signal(
    "measure",
    connect_to=device_setup.logical_signal_groups["q0"].logical_signals["measure_line"],
)
exp_gate.add_signal(
    "acquire",
    connect_to=device_setup.logical_signal_groups["q0"].logical_signals["acquire_line"],
)

We still want the readout section to be played after the gate section. However, using the `play_after` command is not possible anymore, because we cannot specify whether it should be played after the first or the second gate section. Instead, we reserve the drive line in the readout section, to make sure that the sections do not overlap in time:

In [ ]:
rt_loop = AcquireLoopRt(count=2**4)

parent_section = Section(uid="parent_section")

parent_section.add(gate_section)
parent_section.add(gate_section)
parent_section.add(readout_section)

rt_loop.add(parent_section)

exp_gate.add(rt_loop)

In [ ]:
compiled_gate_exp = session.compile(exp_gate)

show_pulse_sheet("Reusing_sections", compiled_gate_exp)